In [ ]:
import setup 

setup.init()

In [ ]:
import os
import io
import contextlib
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

In [ ]:
GOOGLE_GEMINI_API_KEY = os.environ.get("GOOGLE_GEMINI_API_KEY")
assert GOOGLE_GEMINI_API_KEY is not None

In [ ]:
BRIGHT_DATA_SERP_API_KEY = os.environ.get("BRIGHT_DATA_SERP_API_KEY")
assert BRIGHT_DATA_SERP_API_KEY is not None

In [ ]:
from reddit.models import RedditCommunity

In [ ]:
from langchain_brightdata import BrightDataSERP

serp_tool = BrightDataSERP(
    bright_data_api_key=BRIGHT_DATA_SERP_API_KEY,
    search_engine = 'google',
    country = 'in',
    parse_results = True
)

In [ ]:
serp_tool.name

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key = GOOGLE_GEMINI_API_KEY)

In [ ]:
agent_prompt = """
You are an expert assistant that can search the internet to find the best reddit communities for any given topic.
You are also an expert at finding niche communities that discuss the same topic."""

In [ ]:
class RedditCommunitySchema(BaseModel):
    name : str = Field(description= "Formatted name for Rabbit")
    url : str = Field(description="The complete url of the reddit community")
    subreddit_slug :str = Field(description = "The slug of the subbreddit such as r/Python or r/web or r/trending")
    member_count : int | None = Field(description = "Current member count, if available.")

class RedditCommunitiesSchema(BaseModel):
    communitites : list[RedditCommunitySchema] = Field(description="The list of reddit communites.")

In [ ]:
from langchain.agents import create_agent


weather_agent  = create_agent(
    model = model,
    tools = [serp_tool],
    system_prompt = agent_prompt,
    response_format = RedditCommunitiesSchema
)

results = weather_agent.invoke(
    {"messages" : [{"role" : "user", "content" : "Python, JavaScripts, dJango, web"}]},
    strem_mode = "values"
)

In [ ]:
results

In [ ]:
print(results["messages"][-1].content)

In [ ]:
for subreddit in results['structured_response'].communitites:
    print(subreddit.name, subreddit.url, subreddit.subreddit_slug, subreddit.member_count)

In [ ]:
community_data = [x.model_dump() for x in results['structured_response'].communitites]
for data in community_data:
    url = data.pop("url")
    print(data)
    RedditCommunity.objects.update_or_create(url = url, defaults=data)